In [1]:
from uk_covid19 import Cov19API
import pandas as pd
import dataframe_image as dfi

In [3]:
my_ltlas = [
    ['areaType=ltla',
    'areaName=Brighton and Hove'],
    ['areaType=ltla',
    'areaName=Enfield'],
    ['areaType=ltla',
    'areaName=Wolverhampton'],
    ['areaType=ltla',
    'areaName=Mid Sussex'],
    ['areaType=ltla',
    'areaName=Bournemouth, Christchurch and Poole'],
    ['areaType=overview',
    'areaName=United Kingdom']
]

In [4]:
cases_and_vaccines = {
    "date": "date",
    "areaName": "areaName",
    "areaCode": "areaCode",
    "newCasesBySpecimenDate": "newCasesBySpecimenDate",
    "cumCasesBySpecimenDate": "cumCasesBySpecimenDate",
    "cumCasesBySpecimenDateRate": "cumCasesBySpecimenDateRate",
    "cumCasesByPublishDateRate": "cumCasesByPublishDateRate",
    "newCasesByPublishDate": "newCasesByPublishDate",
    "cumCasesByPublishDate": "cumCasesByPublishDate"
}

In [5]:
population = {}

In [1]:
for ltla in my_ltlas:
    api = Cov19API(filters=ltla, structure=cases_and_vaccines,
                  latest_by="cumCasesByPublishDateRate")
    df = api.get_dataframe()
    pop = df['cumCasesByPublishDate'][0] / df['cumCasesByPublishDateRate'][0] * 1e5
    #print(pop)
    ltla_name = ltla[1].split("=")[1]
    #print(ltla_name)
    population[ltla_name] = pop
    latest_date = df['date'][0]

NameError: name 'my_ltlas' is not defined

In [24]:
output_dict = {'Area name': [],
              'Latest published rate': [],
              'Published rate change': [],
              'Latest specimen rate': [],
              'Specimen rate change': []}
output_df = pd.DataFrame(data = output_dict)

In [25]:
for ltla in my_ltlas:
    api = Cov19API(filters=ltla, structure=cases_and_vaccines)
    df = api.get_dataframe()
    ltla_name = ltla[1].split("=")[1]
    # Last 7 days published rate
    published_latest_rate = df.newCasesByPublishDate[:7].sum() / population[ltla_name] * 1e5
    # Prior 7 days published rate
    published_previous_rate = df.newCasesByPublishDate[7:14].sum() / population[ltla_name] * 1e5
    # Latest 7 days specimen date
    specimen_latest_rate = df.newCasesBySpecimenDate[5:12].sum() / population[ltla_name] * 1e5
    # Prior 7 days specimen date
    specimen_previous_rate = df.newCasesBySpecimenDate[12:19].sum() / population[ltla_name] * 1e5
    # Specimen and published rate changes
    published_rate_change = published_latest_rate / published_previous_rate - 1
    specimen_rate_change = specimen_latest_rate / specimen_previous_rate - 1
    # Save results into output_df
    output_df = output_df.append({'Area name': ltla_name,
                                  'Latest published rate': published_latest_rate,
                                  'Latest specimen rate': specimen_latest_rate,
                                  'Published rate change': published_rate_change,
                                  'Specimen rate change': specimen_rate_change},
                                ignore_index=True)
    # print("{} specimen_previous_rate: {}".format(ltla_name, specimen_previous_rate))

In [26]:
styled_output = output_df.sort_values(by=['Latest specimen rate']).style.hide_index().format({
        'Latest published rate': '{:.1f}'.format,
        'Published rate change': '{:,.0%}'.format,
        'Latest specimen rate': '{:.1f}'.format,
        'Specimen rate change': '{:,.0%}'.format,
    })

In [27]:
styled_output

Area name,Latest published rate,Published rate change,Latest specimen rate,Specimen rate change
Mid Sussex,18.5,-20%,16.6,-31%
Brighton and Hove,31.3,-3%,29.2,-10%
Enfield,29.4,-4%,29.4,-9%
"Bournemouth, Christchurch and Poole",33.6,2%,35.4,9%
Wolverhampton,69.5,12%,58.9,-14%
United Kingdom,59.4,-1%,59.5,-2%


![](COVID.png)